In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as Transforms

from tqdm import tqdm
from torchsummary import summary

In [2]:
# Hyper parameter
BATCH_SIZE = 256
LR = 1e-3
EPOCHS = 20

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [4]:
cifar_train = torchvision.datasets.CIFAR10(root='./train/',
                                           train=True,
                                           download=True
                                           )

cifar_test = torchvision.datasets.CIFAR10(root='./test/',
                                          train=False,
                                          download=True
                                          )

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./train/cifar-10-python.tar.gz to ./train/


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./test/cifar-10-python.tar.gz to ./test/


In [5]:
train_transform = Transforms.Compose([Transforms.ToTensor(),
                                      Transforms.Resize((224, 224)),
                                      Transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                                      Transforms.RandomHorizontalFlip()
                                    ])

val_transform = Transforms.Compose([Transforms.ToTensor(),
                                    Transforms.Resize((224, 224)),
                                    Transforms.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                                   ]) 

In [6]:
cifar_train.transform = train_transform
cifar_test.transform = val_transform

In [7]:
trainLoader = torch.utils.data.DataLoader(cifar_train,
                                          batch_size=BATCH_SIZE,
                                          shuffle=True
                                         )
valLoader = torch.utils.data.DataLoader(cifar_test,
                                        batch_size=BATCH_SIZE,
                                        shuffle=True
                                       )

In [8]:
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()

        self.residual_function = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False, stride=stride),
                                               nn.BatchNorm2d(out_channels),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1, bias=False),
                                               nn.BatchNorm2d(out_channels)
                                              )
        self.relu = nn.ReLU()

        self.shortcut = nn.Sequential()
        if stride != 1:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels, kernel_size=1, bias=False, stride=stride),
                                          nn.BatchNorm2d(out_channels)
                                         )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data)
    
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        out = self.relu(x)
        return out

In [9]:
class BottleNeck(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BottleNeck, self).__init__()

        self.residual_function = nn.Sequential(nn.Conv2d(in_channels, out_channels[0], kernel_size=1, bias=False, stride=stride),
                                               nn.BatchNorm2d(out_channels[0]),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels[0], out_channels[1], kernel_size=3, padding=1, bias=False),
                                               nn.BatchNorm2d(out_channels[1]),
                                               nn.ReLU(),
                                               nn.Conv2d(out_channels[1], out_channels[2], kernel_size=1, bias=False),
                                               nn.BatchNorm2d(out_channels[2])
                                              )
        self.relu = nn.ReLU()

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels[2]:
            self.shortcut = nn.Sequential(nn.Conv2d(in_channels, out_channels[2], kernel_size=1, bias=False, stride=stride),
                                          nn.BatchNorm2d(out_channels[2])
                                         )

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.normal_(m.weight.data)
    
    def forward(self, x):
        x = self.residual_function(x) + self.shortcut(x)
        out = self.relu(x)
        return out

In [10]:
class ResNet34(nn.Module):
    def __init__(self):
        super(ResNet34, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU()
                                  )
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2_x = nn.Sequential(BasicBlock(64, 64),
                                     BasicBlock(64, 64),
                                     BasicBlock(64, 64)
                                    )
        self.conv3_x = nn.Sequential(BasicBlock(64, 128, stride=2),
                                     BasicBlock(128, 128),
                                     BasicBlock(128, 128),
                                     BasicBlock(128, 128)
                                    )
        self.conv4_x = nn.Sequential(BasicBlock(128, 256, stride=2),
                                     BasicBlock(256, 256),
                                     BasicBlock(256, 256),
                                     BasicBlock(256, 256),
                                     BasicBlock(256, 256),
                                     BasicBlock(256, 256)
                                    )
        self.conv5_x = nn.Sequential(BasicBlock(256, 512, stride=2),
                                     BasicBlock(512, 512),
                                     BasicBlock(512, 512)
                                    )
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(512, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.flatten(start_dim=1)
        out = self.fc(x)
        return out

In [11]:
resnet34 = ResNet34().to(device)

In [12]:
summary(resnet34, (3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5           [-1, 64, 55, 55]          36,864
       BatchNorm2d-6           [-1, 64, 55, 55]             128
              ReLU-7           [-1, 64, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]          36,864
       BatchNorm2d-9           [-1, 64, 55, 55]             128
             ReLU-10           [-1, 64, 55, 55]               0
       BasicBlock-11           [-1, 64, 55, 55]               0
           Conv2d-12           [-1, 64, 55, 55]          36,864
      BatchNorm2d-13           [-1, 64, 55, 55]             128
             ReLU-14           [-1, 64,

In [13]:
class ResNet50(nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()

        self.conv1 = nn.Sequential(nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3),
                                   nn.BatchNorm2d(64),
                                   nn.ReLU()
                                  )
        self.max_pool = nn.MaxPool2d(kernel_size=3, stride=2)
        self.conv2_x = nn.Sequential(BottleNeck(64, [64, 64, 256]),
                                     BottleNeck(256, [64, 64, 256]),
                                     BottleNeck(256, [64, 64, 256])
                                    )
        self.conv3_x = nn.Sequential(BottleNeck(256, [128, 128, 512], stride=2),
                                     BottleNeck(512, [128, 128, 512]),
                                     BottleNeck(512, [128, 128, 512]),
                                     BottleNeck(512, [128, 128, 512])
                                    )
        self.conv4_x = nn.Sequential(BottleNeck(512, [256, 256, 1024], stride=2),
                                     BottleNeck(1024, [256, 256, 1024]),
                                     BottleNeck(1024, [256, 256, 1024]),
                                     BottleNeck(1024, [256, 256, 1024]),
                                     BottleNeck(1024, [256, 256, 1024]),
                                     BottleNeck(1024, [256, 256, 1024])
                                    )
        self.conv5_x = nn.Sequential(BottleNeck(1024, [512, 512, 2048], stride=2),
                                     BottleNeck(2048, [512, 512, 2048]),
                                     BottleNeck(2048, [512, 512, 2048])
                                    )
        self.avg_pool = nn.AdaptiveAvgPool2d((1,1))
        self.fc = nn.Linear(2048, 10)


    def forward(self, x):
        x = self.conv1(x)
        x = self.max_pool(x)
        x = self.conv2_x(x)
        x = self.conv3_x(x)
        x = self.conv4_x(x)
        x = self.conv5_x(x)
        x = self.avg_pool(x)
        x = x.flatten(start_dim=1)
        out = self.fc(x)
        return out

In [14]:
resnet50 = ResNet50().to(device)

In [15]:
summary(resnet50, (3, 224, 224), device=device)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 55, 55]               0
            Conv2d-5           [-1, 64, 55, 55]           4,096
       BatchNorm2d-6           [-1, 64, 55, 55]             128
              ReLU-7           [-1, 64, 55, 55]               0
            Conv2d-8           [-1, 64, 55, 55]          36,864
       BatchNorm2d-9           [-1, 64, 55, 55]             128
             ReLU-10           [-1, 64, 55, 55]               0
           Conv2d-11          [-1, 256, 55, 55]          16,384
      BatchNorm2d-12          [-1, 256, 55, 55]             512
           Conv2d-13          [-1, 256, 55, 55]          16,384
      BatchNorm2d-14          [-1, 256,

In [16]:
def train_loop(model, Loader):
    train_loss = 0
    train_acc = 0
    model.train()
    for data, target in tqdm(Loader):
        data, target = data.to(device), target.to(device)
        output = model(data)
        loss = loss_fn(output, target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item() 
        train_acc += (torch.argmax(output, dim=1) == target).sum().item()

    return train_loss / len(Loader), train_acc / len(Loader.dataset)

In [17]:
def val_loop(model, Loader):
    val_loss = 0
    val_acc = 0
    model.eval()
    with torch.no_grad():
        for data, target in tqdm(Loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = loss_fn(output, target)

            val_loss += loss.item()
            val_acc += (torch.argmax(output, dim=1) == target).sum().item()

    return val_loss / len(Loader), val_acc / len(Loader.dataset)

In [18]:
optimizer = torch.optim.Adam(resnet34.parameters(), lr=LR)
loss_fn = nn.CrossEntropyLoss()
 # scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.1)

In [19]:
for epoch in range(1, EPOCHS+1):
    train_loss, train_acc = train_loop(resnet34, trainLoader)
    val_loss, val_acc = val_loop(resnet34, valLoader)

    print(f'\n[[  Epoch {epoch:2d} / {EPOCHS}  ]]\n')
    print(f'Train | Loss {train_loss:.4f}, Accuracy : {train_acc*100:.2f} %')
    print(f'Valid | Loss {val_loss:.4f}, Accuracy : {val_acc*100:.2f} %\n')

    # scheduler.step()

100%|██████████| 40/40 [00:27<00:00,  1.48it/s]



[[  Epoch  1 / 20  ]]

Train | Loss 1.6539, Accuracy : 38.42 %
Valid | Loss 1.4530, Accuracy : 46.85 %



100%|██████████| 40/40 [00:27<00:00,  1.47it/s]



[[  Epoch  2 / 20  ]]

Train | Loss 1.2533, Accuracy : 54.23 %
Valid | Loss 1.3412, Accuracy : 51.97 %



100%|██████████| 40/40 [00:27<00:00,  1.47it/s]


...


[[  Epoch 18 / 20  ]]

Train | Loss 0.0839, Accuracy : 97.13 %
Valid | Loss 1.3355, Accuracy : 72.23 %



100%|██████████| 40/40 [00:26<00:00,  1.49it/s]



[[  Epoch 19 / 20  ]]

Train | Loss 0.0782, Accuracy : 97.36 %
Valid | Loss 1.3929, Accuracy : 72.47 %



100%|██████████| 40/40 [00:26<00:00,  1.49it/s]


[[  Epoch 20 / 20  ]]

Train | Loss 0.0666, Accuracy : 97.68 %
Valid | Loss 1.1453, Accuracy : 76.00 %

